<a href="https://colab.research.google.com/github/dineshgumray/ActiveLearning/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install small-text[transformers]  # use "small-text" without "[transformers]" if you want to work on the CPU only
%pip install datasets
%pip install matplotlib

In [ ]:
import datasets
import logging
import numpy as np


# disables the progress bar for notebooks: https://github.com/huggingface/datasets/issues/2651
datasets.logging.get_verbosity = lambda: logging.NOTSET

raw_dataset = datasets.load_dataset('rotten_tomatoes')
num_classes = np.unique(raw_dataset['train']['label']).shape[0]

print('First 10 training samples:\n')
for i in range(10):
    print(raw_dataset['train']['label'][i], ' ', raw_dataset['train']['text'][i])

In [3]:
import transformers
from transformers import AutoTokenizer

transformers.logging.get_verbosity = lambda: logging.NOTSET


transformer_model_name = 'bert-base-uncased'
#'google/electra-base-discriminator'

tokenizer = AutoTokenizer.from_pretrained(
    transformer_model_name
)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [4]:
from small_text.integrations.transformers.datasets import TransformersDataset


def get_transformers_dataset(tokenizer, data, labels, max_length=60):

    data_out = []

    for i, doc in enumerate(data):
        encoded_dict = tokenizer.encode_plus(
            doc,
            add_special_tokens=True,
            padding='max_length',
            max_length=max_length,
            return_attention_mask=True,
            return_tensors='pt',
            truncation='longest_first'
        )

        data_out.append((encoded_dict['input_ids'], encoded_dict['attention_mask'], labels[i]))

    return TransformersDataset(data_out)


train = get_transformers_dataset(tokenizer, raw_dataset['train']['text'], raw_dataset['train']['label'])
test = get_transformers_dataset(tokenizer, raw_dataset['test']['text'], raw_dataset['test']['label'])

In [44]:
test1 = get_transformers_dataset(tokenizer,["effective but too-tepid biopic"], [0])

In [45]:
test1

In [ ]:
from small_text.active_learner import PoolBasedActiveLearner

from small_text.initialization import random_initialization_balanced
from small_text.integrations.transformers import TransformerModelArguments
from small_text.integrations.transformers.classifiers.factories import TransformerBasedClassificationFactory
from small_text.query_strategies import PredictionEntropy
from small_text.integrations.transformers import TransformerModelArguments


# simulates an initial labeling to warm-start the active learning process
def initialize_active_learner(active_learner, y_train):
    

    x_indices_initial = random_initialization_balanced(y_train, n_samples=10)
    y_initial = y_train[x_indices_initial]

    active_learner.initialize_data(x_indices_initial, y_initial)

    return x_indices_initial

transformer_model = TransformerModelArguments(transformer_model_name)
clf_factory = TransformerBasedClassificationFactory(transformer_model, 
                                                    num_classes, 
                                                    kwargs=dict({'device': 'cpu',
                                                                 'mini_batch_size': 30,
                                                                 'early_stopping_no_improvement': -1
                                                                }))
query_strategy = PredictionEntropy()

active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, train)
labeled_indices = initialize_active_learner(active_learner, train.y)

In [47]:
train[[0]]
y_pred = active_learner.classifier.predict(test1[[0]])
y_pred

/usr/local/lib/python3.7/dist-packages/small_text/integrations/pytorch/utils/data.py:35: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  data_set = np.array(data_set, dtype=object, copy=False)


array([1])

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from sklearn import metrics

def evaluate(active_learner, train, test):

    y_pred = active_learner.classifier.predict(train)
    y_pred_prob = active_learner.classifier.predict_proba(train)
    train_acc = accuracy_score(y_pred, train.y)
    fpr, tpr, th = metrics.roc_curve(train.y, y_pred_prob[:,1])
    train_auc = metrics.auc(fpr, tpr)
    log_loss = metrics.log_loss(train.y, y_pred_prob)

    y_pred_test = active_learner.classifier.predict(test)
    y_pred_test_prob = active_learner.classifier.predict_proba(test)  
    test_acc = accuracy_score(y_pred_test, test.y)
    fpr, tpr, th = metrics.roc_curve(test.y, y_pred_test_prob[:,1])
    test_auc = metrics.auc(fpr, tpr)
    log_loss = metrics.log_loss(test.y, y_pred_test_prob)

    print('Train accuracy: {:.2f}'.format(train_acc))
    print('Test accuracy: {:.2f}'.format(test_acc))

    return (train_acc, test_acc, train_auc, test_auc)


res = tuple()
results = defaultdict(list)
res = evaluate(active_learner, train[labeled_indices], test)
results["train"].append(res[0])
results["test"].append(res[1])
results["train_auc"].append(res[2])
results["test_auc"].append(res[3])

/usr/local/lib/python3.7/dist-packages/small_text/integrations/pytorch/utils/data.py:35: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  data_set = np.array(data_set, dtype=object, copy=False)
/usr/local/lib/python3.7/dist-packages/small_text/integrations/pytorch/utils/data.py:35: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain th

Train accuracy: 0.60
Test accuracy: 0.50


In [ ]:
num_queries = 1   

for i in range(num_queries):
    # ...where each iteration consists of labelling 1000 samples
    q_indices = active_learner.query(num_samples=1000)

    # Simulate user interaction here. Replace this for real-world usage.
    y = train.y[q_indices]

    # Return the labels for the current query to the active learner.
    active_learner.update(y)

    labeled_indices = np.concatenate([q_indices, labeled_indices])

    print('Iteration #{:d} ({} samples)'.format(i, len(labeled_indices)))

    res = evaluate(active_learner, train[labeled_indices], test)
    results["train"].append(res[0])
    results["test"].append(res[1])
    results["train_auc"].append(res[2])
    results["test_auc"].append(res[3])

/usr/local/lib/python3.7/dist-packages/small_text/integrations/pytorch/utils/data.py:35: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  data_set = np.array(data_set, dtype=object, copy=False)


In [6]:
import pickle
pickle.dump(active_learner, open('model0.pkl', 'wb'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f = open("/content/drive/MyDrive/rotten_tomatoes_bert_PE_log1.txt", "a")

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from sklearn import metrics

num_queries = 2


def evaluate(active_learner, train, test):

    y_pred = active_learner.classifier.predict(train)
    y_pred_prob = active_learner.classifier.predict_proba(train)
    train_acc = accuracy_score(y_pred, train.y)
    fpr, tpr, th = metrics.roc_curve(train.y, y_pred_prob[:,1])
    train_auc = metrics.auc(fpr, tpr)
    log_loss = metrics.log_loss(train.y, y_pred_prob)

    f.write("\n############### Train Classifier ########################\n")
    f.write(classification_report(train.y, y_pred))
    f.write("\n############### Train Confusion Matrix ##################\n")
    cm = confusion_matrix(train.y, y_pred)
    f.write(str(cm))
    f.write("\nTrain accuracy: ")
    f.write(str(train_acc))
    f.write("\nTrain AUC: ")
    f.write(str(train_auc))
    f.write("\nfpr: {}  tpr: {}   threshold: {}  Loss Fun: {}".format(
        str(fpr),str(tpr),str(th),str(log_loss)))

    y_pred_test = active_learner.classifier.predict(test)
    y_pred_test_prob = active_learner.classifier.predict_proba(test)  
    test_acc = accuracy_score(y_pred_test, test.y)
    fpr, tpr, th = metrics.roc_curve(test.y, y_pred_test_prob[:,1])
    test_auc = metrics.auc(fpr, tpr)
    log_loss = metrics.log_loss(test.y, y_pred_test_prob)

    f.write("\n############### Test Classifier ########################\n")
    f.write(classification_report(test.y, y_pred_test))
    f.write("\n############### Test Confusion Matrix ##################\n")
    cm = confusion_matrix(test.y, y_pred_test)
    f.write(str(cm))
    f.write("\nTest accuracy: ")
    f.write(str(test_acc))
    f.write("\nTest AUC: ")
    f.write(str(test_auc))
    f.write("\nfpr: {}  tpr: {}   threshold: {}  Loss Fun: {}".format(
        str(fpr),str(tpr),str(th),str(log_loss)))

    f.write("\n############### Train ########################\n")
    f.write(str(train))
    f.write("\n############### Train Data ########################\n")
    f.write(str(y_pred))
    f.write("\n############### Train Data Prob ########################\n")
    f.write(str(y_pred_prob))
    f.write("\n############### Test Data ########################\n")
    f.write(str(y_pred_test))
    f.write("\n############### Test Data Prob ########################\n")
    f.write(str(y_pred_test_prob))


    print('Train accuracy: {:.2f}'.format(train_acc))
    print('Test accuracy: {:.2f}'.format(test_acc))
    print('Train LF: {:.4f}'.format(metrics.log_loss(train.y, y_pred)))
    print('Test LF: {:.4f}'.format(metrics.log_loss(test.y, y_pred_test)))

    return (train_acc, test_acc, train_auc, test_auc)


res = tuple()
results = defaultdict(list)
res = evaluate(active_learner, train[labeled_indices], test)
results["train"].append(res[0])
results["test"].append(res[1])
results["train_auc"].append(res[2])
results["test_auc"].append(res[3])
    
for i in range(num_queries):
    # ...where each iteration consists of labelling 1000 samples
    q_indices = active_learner.query(num_samples=1000)

    # Simulate user interaction here. Replace this for real-world usage.
    y = train.y[q_indices]

    # Return the labels for the current query to the active learner.
    active_learner.update(y)

    labeled_indices = np.concatenate([q_indices, labeled_indices])

    print('Iteration #{:d} ({} samples)'.format(i, len(labeled_indices)))
    f.write("\n###########################################################\n")
    f.write("\n###########################################################\n")
    f.write("Iteration ")
    f.write(str(i))
    res = evaluate(active_learner, train[labeled_indices], test)
    results["train"].append(res[0])
    results["test"].append(res[1])
    results["train_auc"].append(res[2])
    results["test_auc"].append(res[3])

Train accuracy: 0.93
Test accuracy: 0.83
Train LF: 2.5904
Test LF: 6.0266


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Iteration #0 (2000 samples)
Train accuracy: 0.92
Test accuracy: 0.83
Train LF: 2.6250
Test LF: 5.8321


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Iteration #1 (3000 samples)
Train accuracy: 0.79
Test accuracy: 0.81
Train LF: 7.3568
Test LF: 6.5125


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Iteration #5 (7000 samples)
Train accuracy: 0.90
Test accuracy: 0.84
Train LF: 3.6217
Test LF: 5.4433


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Iteration #6 (8000 samples)
Train accuracy: 0.91
Test accuracy: 0.84
Train LF: 3.2380
Test LF: 5.4757


PoolExhaustedException: ignored

In [ ]:

f.close()